# Exercise 6

## SVM & Regularization


For this homework we consider a set of observations on a number of red and white wine varieties involving their chemical properties and ranking by tasters. Wine industry shows a recent growth spurt as social drinking is on the rise. The price of wine depends on a rather abstract concept of wine appreciation by wine tasters, opinion among whom may have a high degree of variability. Pricing of wine depends on such a volatile factor to some extent. Another key factor in wine certification and quality assessment is physicochemical tests which are laboratory-based and takes into account factors like acidity, pH level, presence of sugar and other chemical properties. For the wine market, it would be of interest if human quality of tasting can be related to the chemical properties of wine so that certification and quality assessment and assurance process is more controlled.

Two datasets are available of which one dataset is on red wine and have 1599 different varieties and the other is on white wine and have 4898 varieties. All wines are produced in a particular area of Portugal. Data are collected on 12 different properties of the wines one of which is Quality, based on sensory data, and the rest are on chemical properties of the wines including density, acidity, alcohol content etc. All chemical properties of wines are continuous variables. Quality is an ordinal variable with possible ranking from 1 (worst) to 10 (best). Each variety of wine is tasted by three independent tasters and the final rank assigned is the median rank given by the tasters.

A predictive model developed on this data is expected to provide guidance to vineyards regarding quality and price expected on their produce without heavy reliance on volatility of wine tasters.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [3]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
4819,5.7,0.400,0.35,5.1,0.026,17.0,113.0,0.99052,3.18,0.67,12.4,6,white
4130,6.8,0.290,0.32,1.8,0.032,18.0,130.0,0.99095,3.05,0.62,11.2,6,white
4872,6.0,0.420,0.41,12.4,0.032,50.0,179.0,0.99622,3.14,0.60,9.7,5,white
845,7.3,0.210,0.29,1.6,0.034,29.0,118.0,0.99170,3.30,0.50,11.0,8,white
4930,8.3,0.655,0.12,2.3,0.083,15.0,113.0,0.99660,3.17,0.66,9.8,5,red


# Exercise 6.1

Show the frecuency table of the quality by type of wine

In [4]:
data.groupby(['type','quality'])['quality'].agg({'Frequency':'count'})

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


Frequency
type  quality           
red   3               10
      4               53
      5              681
      6              638
      7              199
      8               18
white 3               20
      4              163
      5             1457
      6             2198
      7              880
      8              175
      9                5

In [5]:
data['label']=np.where(data['quality']<=6, 0, 1)
data['label'] = data['label'].astype('category')
data['type'] = data['type'].astype('category')
data['quality'] = data['quality'].astype('int')

data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type,label
3802,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,5,white,0
4082,5.9,0.230,0.28,8.6,0.046,37.0,142.0,0.99432,3.23,0.53,10.6,6,white,0
4109,5.0,0.255,0.22,2.7,0.043,46.0,153.0,0.99238,3.75,0.76,11.3,6,white,0
6444,7.0,0.570,0.02,2.0,0.072,17.0,26.0,0.99575,3.36,0.61,10.2,5,red,0
6379,8.2,0.280,0.60,3.0,0.104,10.0,22.0,0.99828,3.39,0.68,10.6,5,red,0


# SVM

# Exercise 6.2

* Standarized the features (not the quality)
* Create a binary target for each type of wine
* Create two Linear SVM's for the white and red wines, repectively.


In [6]:
data.columns.values
import warnings
from sklearn.svm import SVC # "Support Vector Classifier"
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.model_selection import train_test_split  
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

scalerw = StandardScaler()
scalerr = StandardScaler()
warnings.filterwarnings('ignore')
xdf_w = data[data['type']=='white']
ydf_w = xdf_w[['label']]
xdf_w = xdf_w.drop(['type'], axis=1)
xdf_w = xdf_w.drop(['label'], axis=1)

xdf_r = data[data['type']=='red']
ydf_r = xdf_r[['label']]
xdf_r = xdf_r.drop(['type'], axis=1)
xdf_r = xdf_r.drop(['label'], axis=1)

In [7]:
data[data['type']=='white'].groupby(['label'])['quality'].agg({'Frequency':'count'})

,Frequency
label,
0,3838
1,1060


In [8]:
data[data['type']=='red'].groupby(['label'])['quality'].agg({'Frequency':'count'})

,Frequency
label,
0,1382
1,217


In [9]:
xdf_r = xdf_r.drop(['quality'], axis=1)
xdf_w = xdf_w.drop(['quality'], axis=1)
scalerw.fit(xdf_w)
xdf_w = scalerw.transform(xdf_w)
scalerr.fit(xdf_r)
xdf_r = scalerr.transform(xdf_r)
X_trainw, X_testw, y_trainw, y_testw = train_test_split(xdf_w, ydf_w, test_size = 0.30)
X_trainr, X_testr, y_trainr, y_testr = train_test_split(xdf_r, ydf_r, test_size = 0.30)

# Exercise 6.3

Test the two SVM's using the different kernels (‘poly’, ‘rbf’, ‘sigmoid’)


In [10]:
#Kernel Lineal
print('Kernel Lineal para el dataset tipo white')

clf1 = SVC(kernel='linear')
clf1.fit(X_trainw, y_trainw)
y_clf1 = clf1.predict(X_testw)
print(confusion_matrix(y_testw,y_clf1))  
print(classification_report(y_testw,y_clf1))  
print(clf1.score(X_testw, y_testw))

print('Kernel Lineal para el dataset tipo red')

clf2 = SVC(kernel='linear')
clf2.fit(X_trainr, y_trainr)
y_clf2 = clf2.predict(X_testr)
print(confusion_matrix(y_testr,y_clf2))  
print(classification_report(y_testr,y_clf2))  
print(clf2.score(X_testr, y_testr))

#Kernel Poly
print('Kernel Poly para el dataset tipo white')

clf1p = SVC(kernel='poly')
clf1p.fit(X_trainw, y_trainw)
y_clf1p = clf1p.predict(X_testw)
print(confusion_matrix(y_testw,y_clf1p))  
print(classification_report(y_testw,y_clf1p))  
print(clf1p.score(X_testw, y_testw))

print('Kernel Poly para el dataset tipo red')
clf2p = SVC(kernel='poly')
clf2p.fit(X_trainr, y_trainr)
y_clf2p = clf2p.predict(X_testr)
print(confusion_matrix(y_testr,y_clf2p))  
print(classification_report(y_testr,y_clf2p))  
print(clf2p.score(X_testr, y_testr))


#Kernel rbf
print('Kernel RBF para el dataset tipo white')
clf1r = SVC(kernel='rbf')
clf1r.fit(X_trainw, y_trainw)
y_clf1r = clf1r.predict(X_testw)
print(confusion_matrix(y_testw,y_clf1r))  
print(classification_report(y_testw,y_clf1r))  
print(clf1r.score(X_testw, y_testw))

print('Kernel RBF para el dataset tipo red')
clf2r = SVC(kernel='rbf')
clf2r.fit(X_trainr, y_trainr)
y_clf2r = clf2r.predict(X_testr)
print(confusion_matrix(y_testr,y_clf2r))  
print(classification_report(y_testr,y_clf2r))  
print(clf2r.score(X_testr, y_testr))

#Kernel sigmoid

print('Kernel Sigmoid para el dataset tipo white')
clf1s = SVC(kernel='sigmoid')
clf1s.fit(X_trainw, y_trainw)
y_clf1s = clf1s.predict(X_testw)
print(confusion_matrix(y_testw,y_clf1s))  
print(classification_report(y_testw,y_clf1s))  
print(clf1s.score(X_testw, y_testw))

print('Kernel Sigmoid para el dataset tipo red')
clf2s = SVC(kernel='sigmoid')
clf2s.fit(X_trainr, y_trainr)
y_clf2s = clf2s.predict(X_testr)
print(confusion_matrix(y_testr,y_clf2s))  
print(classification_report(y_testr,y_clf2s))  
print(clf2s.score(X_testr, y_testr))


Kernel Lineal para el dataset tipo white
[[1142    0]
 [ 328    0]]
              precision    recall  f1-score   support

           0       0.78      1.00      0.87      1142
           1       0.00      0.00      0.00       328

   micro avg       0.78      0.78      0.78      1470
   macro avg       0.39      0.50      0.44      1470
weighted avg       0.60      0.78      0.68      1470

0.7768707482993197
Kernel Lineal para el dataset tipo red
[[406   0]
 [ 74   0]]
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       406
           1       0.00      0.00      0.00        74

   micro avg       0.85      0.85      0.85       480
   macro avg       0.42      0.50      0.46       480
weighted avg       0.72      0.85      0.78       480

0.8458333333333333
Kernel Poly para el dataset tipo white
[[1129   13]
 [ 273   55]]
              precision    recall  f1-score   support

           0       0.81      0.99      0.89      1142
   

Luego de correr las confusion matrix de cada kernel se evidencia un buen ajuste para los kernels:

#### Tipo white:

* Kernel Poly
* Kernel RBF

#### Tipo red:

* kernel RBF
* kernel Poly

# Exercise 6.4
Using the best SVM find the parameters that gives the best performance

'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]

Para este ejercicio se selecciona el kernel RBF como el mejor SVM 

In [11]:
#Tipo white
Cp = [0.1, 1, 10, 100, 1000]
gammap = [0.01, 0.001, 0.0001]
a=1
l=[]
print('Tipo White')
for i in Cp:
    for j in gammap:
        modl = SVC(kernel='rbf', C= i, gamma = j)
        modl.fit(X_trainw, y_trainw)
        ypred = modl.predict(X_testw)
        score = modl.score(X_testw, y_testw)
        print(str(a)+ '. C: ' + str(i) + ', Gamma: ' + str(j) +  ', Score: ' + str(score))
        a+=1
        l.append([a,i,j,score])
l = pd.DataFrame(l, columns = ['iteration','C','Gamma', 'Score'])
print('Observación con el mejor score: ')
print(l.loc[l['Score'].idxmax()])

#Tipo Red
print(' ')
a=1
l=[]
print('Tipo Red')
for i in Cp:
    for j in gammap:
        modl = SVC(kernel='rbf', C= i, gamma = j)
        modl.fit(X_trainr, y_trainr)
        ypred = modl.predict(X_testr)
        score = modl.score(X_testr, y_testr)
        print(str(a)+ '. C: ' + str(i) + ', Gamma: ' + str(j) +  ', Score: ' + str(score))
        a+=1
        l.append([a,i,j,score])
l = pd.DataFrame(l, columns = ['iteration','C','Gamma', 'Score'])
print('Observación con el mejor score: ')
print(l.loc[l['Score'].idxmax()])



Tipo White
1. C: 0.1, Gamma: 0.01, Score: 0.7768707482993197
2. C: 0.1, Gamma: 0.001, Score: 0.7768707482993197
3. C: 0.1, Gamma: 0.0001, Score: 0.7768707482993197
4. C: 1, Gamma: 0.01, Score: 0.7802721088435374
5. C: 1, Gamma: 0.001, Score: 0.7768707482993197
6. C: 1, Gamma: 0.0001, Score: 0.7768707482993197
7. C: 10, Gamma: 0.01, Score: 0.8156462585034013
8. C: 10, Gamma: 0.001, Score: 0.7768707482993197
9. C: 10, Gamma: 0.0001, Score: 0.7768707482993197
10. C: 100, Gamma: 0.01, Score: 0.827891156462585
11. C: 100, Gamma: 0.001, Score: 0.7925170068027211
12. C: 100, Gamma: 0.0001, Score: 0.7768707482993197
13. C: 1000, Gamma: 0.01, Score: 0.8272108843537415
14. C: 1000, Gamma: 0.001, Score: 0.8197278911564626
15. C: 1000, Gamma: 0.0001, Score: 0.7768707482993197
Observación con el mejor score: 
iteration     11.000000
C            100.000000
Gamma          0.010000
Score          0.827891
Name: 9, dtype: float64
 
Tipo Red
1. C: 0.1, Gamma: 0.01, Score: 0.8458333333333333
2. C: 0.1, 

# Exercise 6.5

Compare the results with other methods

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
a=1
print('Tipo White')
l=[]
for i in Cp:
        logregw = LogisticRegression(solver='liblinear',C=i)
        logregw.fit(X_trainw, y_trainw)
        ypredw = logregw.predict(X_testw)
        score = logregw.score(X_testw, y_testw)
        ff = f1_score(y_testw, ypredw, average='macro')
        print(str(a)+ '. C: ' + str(i) +   ', Score: ' + str(score)+' F1 score: '+str(ff))
        l.append([a,i,score,ff])
        a+=1
l = pd.DataFrame(l, columns = ['iteration','C','Score','F1 Score'])
print('Observación con el mejor score: ')
print(l.loc[l['Score'].idxmax()])

#Tipo Red
print(' ')
a=1
l=[]
print('Tipo Red')
for i in Cp:
        logregr = LogisticRegression(solver='liblinear',C=i)
        logregr.fit(X_trainr, y_trainr)
        ypredr = logregr.predict(X_testr)
        score = logregr.score(X_testr, y_testr)
        ff = f1_score(y_testr, ypredr, average='macro')
        print(str(a)+ '. C: ' + str(i) +  ', Score: ' + str(score)+' F1 score: '+str(ff))
        l.append([a,i,score,ff])
        a+=1
l = pd.DataFrame(l, columns = ['iteration','C','Score','F1'])
print('Observación con el mejor score: ')
print(l.loc[l['Score'].idxmax()])



Tipo White
1. C: 0.1, Score: 0.8061224489795918 F1 score: 0.6307935537144926
2. C: 1, Score: 0.8013605442176871 F1 score: 0.6289726993610489
3. C: 10, Score: 0.7993197278911565 F1 score: 0.6270957323034932
4. C: 100, Score: 0.7993197278911565 F1 score: 0.6270957323034932
5. C: 1000, Score: 0.7993197278911565 F1 score: 0.6270957323034932
Observación con el mejor score: 
iteration    1.000000
C            0.100000
Score        0.806122
F1 Score     0.630794
Name: 0, dtype: float64
 
Tipo Red
1. C: 0.1, Score: 0.8666666666666667 F1 score: 0.6606424815518537
2. C: 1, Score: 0.8708333333333333 F1 score: 0.6765779864371413
3. C: 10, Score: 0.8708333333333333 F1 score: 0.6765779864371413
4. C: 100, Score: 0.8708333333333333 F1 score: 0.6765779864371413
5. C: 1000, Score: 0.8708333333333333 F1 score: 0.6765779864371413
Observación con el mejor score: 
iteration    2.000000
C            1.000000
Score        0.870833
F1           0.676578
Name: 1, dtype: float64


# Regularization

# Exercise 6.6


* Train a linear regression to predict wine quality (Continous)

* Analyze the coefficients

* Evaluate the RMSE

In [13]:
from sklearn.linear_model import LinearRegression
linregw = LinearRegression(fit_intercept=False)
linregr = LinearRegression(fit_intercept=False)

x_w = data[data['type']=='white']
x_r = data[data['type']=='red']
y_w = x_w[['quality']]
y_r = x_r[['quality']]
x_w = x_w.drop(['quality', 'type', 'label'], axis=1)
x_r = x_r.drop(['quality', 'type', 'label'], axis=1)

scalerw.fit(x_w)
x_w = scalerw.transform(x_w)
scalerr.fit(x_r)
x_r = scalerr.transform(x_r)
X_trainw, X_testw, y_trainw, y_testw = train_test_split(x_w, y_w, test_size = 0.30)
X_trainr, X_testr, y_trainr, y_testr = train_test_split(x_r, y_r, test_size = 0.30)
linregw.fit(X_trainw, y_trainw)
linregr.fit(X_trainr, y_trainr)
y_predw=linregw.predict(X_testw)
y_predr=linregr.predict(X_testr)

In [14]:
print('Tipo white')
print('MAE:', metrics.mean_absolute_error(y_testw, y_predw))
print('MSE:', metrics.mean_squared_error(y_testw, y_predw))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_testw, y_predw)))
print('Tipo Red')
print('MAE:', metrics.mean_absolute_error(y_testr, y_predr))
print('MSE:', metrics.mean_squared_error(y_testr, y_predr))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_testr, y_predr)))

Tipo white
MAE: 5.8811949226886995
MSE: 35.30777788077848
RMSE: 5.9420348266211365
Tipo Red
MAE: 5.693102177234306
MSE: 33.00639689514137
RMSE: 5.745119397814233


# Exercise 6.7

* Estimate a ridge regression with alpha equals 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

In [15]:
from sklearn.linear_model import Ridge
print('Tipo white: ')
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_testw, y_predw)))
print('Coeficientes de la regresión lineal: ')
print(linregw.coef_)
print()
alpha = [0.1,1]
for i in alpha:
    ridgereg = Ridge(alpha=i, normalize=True)
    ridgereg.fit(X_trainw, y_trainw)
    y_predw = ridgereg.predict(X_testw)
    print('Alpha: ' + str(i)+ ' RMSE: '+ str(np.sqrt(metrics.mean_squared_error(y_testw, y_predw))))
    print(ridgereg.coef_)
    print()
print()
print('Tipo Red: ')
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_testr, y_predr)))
print('Coeficientes de la regresión lineal: ')
print(linregr.coef_)
print()
for i in alpha:
    ridgereg = Ridge(alpha=i, normalize=True)
    ridgereg.fit(X_trainr, y_trainr)
    y_predr = ridgereg.predict(X_testr)
    print('Alpha: ' + str(i)+ ' RMSE: '+ str(np.sqrt(metrics.mean_squared_error(y_testr, y_predr))))
    print(ridgereg.coef_)
    print()

Tipo white: 
RMSE: 5.9420348266211365
Coeficientes de la regresión lineal: 
[[ 0.40239127 -0.15564373 -0.00580592  1.04724846  0.05394599 -0.11278763
   0.18233661 -1.55140399  0.38402605  0.09906177 -0.28869721]]

Alpha: 0.1 RMSE: 0.7742058704135955
[[-0.01325499 -0.16934548  0.00660391  0.18378882 -0.03769504  0.06946547
  -0.03785038 -0.16286496  0.0368686   0.05319908  0.3023521 ]]

Alpha: 1 RMSE: 0.8115171361132123
[[-0.02432859 -0.08569844  0.00930675  0.03357096 -0.0523062   0.0337533
  -0.03556163 -0.07825408  0.01915921  0.02488484  0.15609607]]


Tipo Red: 
RMSE: 5.745119397814233
Coeficientes de la regresión lineal: 
[[ 0.04787297 -0.14570813  0.01319968  0.17857954 -0.33023637 -0.03991155
  -0.01745066  0.07158147  0.0933917   0.02828843  0.38365824]]

Alpha: 0.1 RMSE: 0.6418092210098201
[[ 0.03838211 -0.17528691 -0.00096938  0.03628329 -0.0622132   0.03862581
  -0.10336864 -0.08508184 -0.05000306  0.16348114  0.24476471]]

Alpha: 1 RMSE: 0.6625920752836787
[[ 0.02380914 -0

# Exercise 6.8

* Estimate a lasso regression with alpha equals 0.01, 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

In [16]:
from sklearn.linear_model import Lasso
lasso = [0.01,0.1,1]
print('Tipo white: ')
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_testw, y_predw)))
print('Coeficientes de la regresión lineal: ')
print(linregw.coef_)
print()
for i in lasso:
    lassoreg = Lasso(alpha=i, normalize=True)
    lassoreg.fit(X_trainw, y_trainw)
    y_predw = lassoreg.predict(X_testw)
    print('Alpha: ' + str(i)+ ' RMSE: '+ str(np.sqrt(metrics.mean_squared_error(y_testw, y_predw))))
    print(lassoreg.coef_)
    print()
print()
print('Tipo Red: ')
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_testr, y_predr)))
print('Coeficientes de la regresión lineal: ')
print(linregr.coef_)
print()
for i in lasso:
    lassoreg = Lasso(alpha=i, normalize=True)
    lassoreg.fit(X_trainr, y_trainr)
    y_predr = lassoreg.predict(X_testr)
    print('Alpha: ' + str(i)+ ' RMSE: '+ str(np.sqrt(metrics.mean_squared_error(y_testr, y_predr))))
    print(lassoreg.coef_)
    print()

Tipo white: 
RMSE: 0.8115171361132123
Coeficientes de la regresión lineal: 
[[ 0.40239127 -0.15564373 -0.00580592  1.04724846  0.05394599 -0.11278763
   0.18233661 -1.55140399  0.38402605  0.09906177 -0.28869721]]

Alpha: 0.01 RMSE: 0.9061490392963572
[-0. -0. -0. -0. -0. -0. -0. -0.  0.  0.  0.]

Alpha: 0.1 RMSE: 0.9061490392963572
[-0. -0. -0. -0. -0. -0. -0. -0.  0.  0.  0.]

Alpha: 1 RMSE: 0.9061490392963572
[-0. -0. -0. -0. -0. -0. -0. -0.  0.  0.  0.]


Tipo Red: 
RMSE: 0.6625920752836787
Coeficientes de la regresión lineal: 
[[ 0.04787297 -0.14570813  0.01319968  0.17857954 -0.33023637 -0.03991155
  -0.01745066  0.07158147  0.0933917   0.02828843  0.38365824]]

Alpha: 0.01 RMSE: 0.766179760910322
[ 0.         -0.          0.         -0.         -0.         -0.
 -0.         -0.         -0.          0.          0.05414584]

Alpha: 0.1 RMSE: 0.78987572146548
[ 0. -0.  0.  0. -0. -0. -0. -0. -0.  0.  0.]

Alpha: 1 RMSE: 0.78987572146548
[ 0. -0.  0.  0. -0. -0. -0. -0. -0.  0.  0.]


# Exercise 6.9

* Create a binary target

* Train a logistic regression to predict wine quality (binary)

* Analyze the coefficients

* Evaluate the f1score

In [17]:
from sklearn.metrics import f1_score
logregw = LogisticRegression(solver='liblinear',C=0.1)
logregw.fit(X_trainw, y_trainw)
ypredw = logregw.predict(X_testw)
scorew = logregw.score(X_testw, y_testw)
print('Tipo white:')
print()
print(logregw.coef_)
print('F1 Score: ')
print(f1_score(y_testw, ypredw, average='macro'))

logregr = LogisticRegression(solver='liblinear',C=0.1)
logregr.fit(X_trainr, y_trainr)
ypredr = logregr.predict(X_testr)
scorer = logregw.score(X_testr, y_testr)
print('Tipo Red:')
print()
print(logregr.coef_)
print('F1 Score: ')
print(f1_score(y_testr, ypredr, average='macro'))


Tipo white:

[[ 0.14786309  0.0576165  -0.07592842 -0.01740489  0.09048869  0.19216283
   0.12635963  0.04241879  0.01904409  0.02867498  0.10627339]
 [ 0.17863616  0.46127132 -0.05610668 -0.51522883  0.0474944  -0.58784883
  -0.1480335   0.50314073  0.08069061 -0.06247068 -0.27966954]
 [-0.0622802   0.41173976  0.02214245 -0.44442919 -0.02022393 -0.14596657
   0.15081949  0.3446306  -0.13325935 -0.20398084 -0.94755873]
 [-0.0351204  -0.35123733  0.02063495  0.03116836  0.03612653  0.0670354
  -0.01009293  0.08849867 -0.00208664  0.02213275  0.25338867]
 [ 0.10688502 -0.37393655 -0.11440129  0.61359212 -0.3494193   0.07710284
  -0.07185097 -0.65369635  0.17455349  0.22350053  0.51519277]
 [ 0.05097223 -0.11989038  0.04429199  0.43657305 -0.12465568  0.1844019
  -0.09875051 -0.34015565  0.129206   -0.0688646   0.55146251]
 [ 0.07705624  0.00388888  0.0358892   0.02699377 -0.02991216  0.00689977
  -0.02630116 -0.02377416  0.07853567 -0.00253503  0.07096493]]
F1 Score: 
0.1860669661250173

# Exercise 6.10

* Estimate a regularized logistic regression using:
* C = 0.01, 0.1 & 1.0
* penalty = ['l1, 'l2']
* Compare the coefficients and the f1score

In [18]:
c=[0.01,0.1,1]
penalty = ['l1','l2']
lw=[]
for i in c:
    for j in penalty:
        logreg = LogisticRegression(C=i, penalty=j,solver='liblinear',multi_class='auto')
        logreg.fit(X_trainw, y_trainw)
        ypredw = logreg.predict(X_testw)
        ff = f1_score(y_testw, ypredw, average='macro')
        lw.append([logreg.coef_, ff,i,j])
lw=pd.DataFrame(lw,columns=['Coeficientes', 'F1 Score','C','Penalty'])
lr=[]
for i in c:
    for j in penalty:
        logreg = LogisticRegression(C=i, penalty=j,solver='liblinear',multi_class='auto')
        logreg.fit(X_trainr, y_trainr)
        ypredr = logreg.predict(X_testr)
        ff = f1_score(y_testr, ypredr, average='macro')
        lr.append([logreg.coef_, ff,i,j])  
lr=pd.DataFrame(lr,columns=['Coeficientes', 'F1 Score','C','Penalty'])

In [19]:
lw

,Coeficientes,F1 Score,C,Penalty
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.164132,0.01,l1
1,"[[0.015358464200791415, 0.008528093848146432, ...",0.177797,0.01,l2
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.20597788201046538...",0.187388,0.10,l1
3,"[[0.1478630934630253, 0.05761649770258228, -0....",0.186067,0.10,l2
4,"[[0.5794118365970393, 0.11932008992245796, -0....",0.190443,1.00,l1
5,"[[0.4719545636694146, 0.12434153470324606, -0....",0.191393,1.00,l2


In [20]:
lr

,Coeficientes,F1 Score,C,Penalty
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.198849,0.01,l1
1,"[[0.003777706186641561, 0.04095656743316204, 0...",0.202363,0.01,l2
2,"[[0.0, 0.09198882631614978, 0.0, 0.0, 0.0, 0.0...",0.211076,0.10,l1
3,"[[0.026664689847972555, 0.24399616948614938, 0...",0.227843,0.10,l2
4,"[[0.0, 1.4983890966721505, 0.23479144833102675...",0.329170,1.00,l1
5,"[[0.04684865024368892, 0.91192505380389, 0.379...",0.261794,1.00,l2
